In [11]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

from scipy.io import mmread

sys.path.append(os.path.abspath("./src"))

import utility as ut
import hessenberg as hg
from variables import *
from qr import QR


pd.set_option("display.max_columns", 10)
pd.set_option("display.width", 1000)
pd.set_option("display.precision", 12)
print("Setup compelete.")


Setup compelete.


# Hessenberg Transform

The Hessenberg transform is a similarity transformation used to reduce a complex matrix to hessenberg form. The following implementation performs the transform using *householder vectors*. The following cells check that for a given matrix $M \in \mathbb{C}^{n \times n}$,
$$M = UHU^{*}$$
where, H is the hessenberg form of $M$ and $U$ is a unitary matrix.

The unittests present in `/tests` check for the equivalence of the eigenvalues of $M$ and $H$. 

## Complex Matrices

In [4]:
a = -20
b = 50
n = 5
m = ut.complex_matrix(n, a, b)
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.conj().T - m, np.zeros((n, n)))}")


Original matrix:
                                   0                                 1                                 2                                 3                                 4
0 -18.828269712986+32.901384313651j  37.786437667179+23.451221934938j  -1.412707331746+37.784628551176j  20.509967602919+46.376168566229j -17.534942834591-11.149463087496j
1  44.501117558643+18.287925638198j -12.383192777215+45.745708101596j  -0.282597816769+46.808873144995j -19.682840002050+15.448792596637j  22.857147839597-13.067357796161j
2  -1.399528464431+18.819472975321j  -14.503902631266-1.579406884332j  24.607462504695+44.343732998387j   48.778610148204-8.030431351372j  -17.440987818192-7.285472143371j
3  8.4360584587990+3.2196945200300j   14.022496905022+4.780312310990j -10.522483170921+14.268332848905j   45.767784159189+3.011616583698j   19.309001709631+3.533840062368j
4   45.022702205448+8.862750749672j  -12.517190654014-8.372963244843j  11.295126736544+14.963712976262j  48.609649012251-1

## Real Matrices

### Random Matrices

In [5]:
a = -20
b = 50
n = 5
m = (b - a) * np.random.default_rng().random((n, n)) + a
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((n, n)))}")

Original matrix:
                  0                1                2                3                4
0  -8.231157281977  15.785892986453  13.221022642740  25.571700608075   1.958510472212
1  14.793812408463  49.476576785424   4.313167331817  32.565371512466  46.609064191689
2   4.309667453821  15.327729910082  27.979778029236  30.763995452227  -6.423218013874
3  34.423191299996   5.701647134297 -19.074713468268  48.987038398312  36.194928167937
4  10.759657530223  -9.351780244926  33.567790702914  25.286227288989  38.141006709085
Hessenberg transformed:
                  0                1                2                3                4
0  -8.231157281977 -30.389197387796  -7.450868767431  -6.591552864001  -7.698711576332
1 -39.219337686697  82.075908800405  -0.878043712643 -12.161146702485 -37.933364915704
2   0.000000000000  44.398402014907  38.722585915113   9.721904312394  -1.137913310155
3   0.000000000000   0.000000000000 -12.810478070155  22.488986407369  25.035615434419


### Matrix Market

In [6]:
files = ["west0381", "blckhole"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	h, u = hg.hessenberg_transform(m) 
	print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((m.shape[0], m.shape[0])))}")

Is the transformation similar: True
Is the transformation similar: True


# QR 

For a given matrix $M \in \mathbb{C}^{n\times n}$, in general, the $QR$ algorithm seeks to perform the following iteration:
* $Q_kR_k := M_k$
* $M_{k + 1} := R_kQ_k$

This algorithm can be made more stable and efficient in two ways. The first is to use $M$ is hessenberg form and the second is use to use shifts. When $H$ (hessenberg form of $M$), is used, the $QR$ decompisition of $H$ can be procedurally generated using Givens rotation matrices, $G$ (see the documentation for explanation and generation). The generation of the QR decomposition and then the subsequent formation of $RQ$ takes place as follows
* $R := G_1 G_2 \dots G_k H.$
* $Q := G_1 G_2 \dots G_k.$
* $H_{\text{new}} := R G_{k}^{*} G_{k - 1}^{*} \dots G_{1}^{*} = RQ.$

where $k \leq n - 2$.


## Wilkinson Shift

The Wilkinson shift employs stable shifts to accelerate convergence of the $QR$ hessenberg algorithm. In general the shifted algorithm looks as follows 
* $Q_kR_k := M_k - \sigma I$
* $M_{k + 1} := R_kQ_k + \sigma I$

The shift $\sigma$ is calculated as detailed in the documentation. The $QR$ decomposition and subsequent formation of $RQ$ is done using the hessenberg form of $M$ and Givens matrices as shown above.

### Complex Matrices

In [10]:
a = -20
b = 50
n = 5
tol = 1e-8
m = ut.complex_matrix(n, a, b)
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")


                Eigenvalues (Numpy)
0  99.917042595485+93.478114421454j
1   45.230266247410+7.871860942562j
2 -11.887599179127-34.047639755158j
3 -18.439477297520+10.127609490804j
4 -26.068468364964+45.972981280689j
               Eigenvalues (Script)
0  99.917042595485+93.478114421454j
1   45.230266247410+7.871860942562j
2 -11.887599179127-34.047639755158j
3 -18.439477297520+10.127609490804j
4 -26.068468364964+45.972981280688j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True
First eigenvector: [ 0.01841364+0.13003535j -0.13860175+0.03950605j -0.32434983-0.1526121j
 -0.6007521 +0.29074996j  0.44996302-0.43079225j]
Eigenvector check: [ 0.17096843+1.00638787j  0.45030444-1.44035828j  1.70889685-0.84949751j
  1.02859368-0.96529076j -0.961102  +0.81167621j]


In [15]:
# Generate a symmetric matrix.
a = -20
b = 50
n = 5
tol = 1e-8
m = ut.complex_matrix(n, a, b)
m = 0.5 * (m + m.T) 
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")
    
# Check if the matrix isi symmetric.
# print(f"Is the matrix symmetric: {np.allclose(m, m.T)}")

rand_choice_eigvec = np.random.default_rng().integers(low = 0, high = n)
eig_vec = u[:, rand_choice_eigvec]
print(f"First eigenvector: {eig_vec}")
print(f"Eigenvector check: {(qr_alg.H @ eig_vec) / np.diag(r)[rand_choice_eigvec]}")

                Eigenvalues (Numpy)
0  82.834131061601+91.284265780251j
1  51.199410674230-17.321804969126j
2  10.609784674471+19.642699979050j
3   9.051827887740-16.880665822175j
4 -32.077881191663-20.716229254387j
               Eigenvalues (Script)
0  82.834131061601+91.284265780251j
1  51.199410674230-17.321804969126j
2  10.609784674471+19.642699979050j
3   9.051827887740-16.880665822175j
4 -32.077881191663-20.716229254387j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True
First eigenvector: [-0.3522261 -0.63977945j  0.24502904+0.39834692j  0.26831945+0.07077897j
  0.33186447-0.18534107j  0.16063764-0.02459192j]
Eigenvector check: [-0.1101313 -0.08833176j  0.60424206+0.65124618j  0.50060955-0.07294095j
  0.41788974+0.46470901j  0.05738401+0.0087919j ]


### Real Matrices

#### Random Matrices

In [14]:
a = -20
b = 50
n = 5
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 100)
eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0   83.869670863615+0.000000000000j
1  29.330356649093+16.228147596952j
2  29.330356649093-16.228147596952j
3  -21.698910577692+0.000000000000j
4  -38.477305285256+0.000000000000j
               Eigenvalues (Script)
0   83.869670863615+0.000000000000j
1  29.330356649093+16.228147596951j
2  29.330356649093-16.228147596951j
3  -21.698910577692+0.000000000000j
4  -38.477305285256+0.000000000000j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [13]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)
tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 100)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


#### Matrix Market

In [22]:
files = ["gre__115"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	tol = 1e-8
	qr_alg = QR(m)
	u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
	eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
	eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
	b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
	print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
	if not b:
		print(f"For matrix {file}")
		print(f"Number of mismatched eigenvalues: {mm.shape[0]}")
		print(f"Average absolute difference in mismatched values {mm['Difference'].mean()}")
		# with open("output_mm.txt", "w") as f:
		# 	f.write(f"{mm.to_string()}")
   


Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


## Double Shift (Inefficient)
For real matrices that have complex eigenvalues (that come in complex conjugate pairs) the shift as explained before can be collapsed into one step (explained in the documentation).

#### Random Matrices

In [14]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)

tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 200)
eigs = np.sort(np.linalg.eig(m)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [4]:
a = -20
b = 50
n = 5
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 200)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0   51.419436746842-0.000000000000j
1  22.167285953793+32.133366474526j
2  22.167285953793-32.133366474526j
3  7.7293981794390+8.8070459431140j
4  7.7293981794390-8.8070459431140j
               Eigenvalues (Script)
0   51.419436746843+0.000000000000j
1  22.167285953793+32.133366474526j
2  22.167285953793-32.133366474526j
3  7.7293981794390+8.8070459431140j
4  7.7293981794390-8.8070459431140j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


# Convergence

## Wilkinson Shift

The Wilkinson Shift (infact the shifted $QR$ algorithm in general) should converge quadratically. The alogorithm relies on the fact that the last subdiagonal element $h_{k - 1, k - 2}$ converges to 0, where $k$ is the 'active' size of the hessenberg matrix H. We expect that 
$$\frac{h^{i + 1}_{k - 1, k - 2}}{h^{i}_{k - 1, k - 2}} \approx \frac{(h^{i}_{k - 1, k - 2})^2}{h^{i}_{k - 1, k - 2}} \approx \;\text{constant}\; (< 1) $$
as $i \rightarrow \infty$. 

In [224]:
m = np.random.default_rng().random((5, 5))

tol = 1e-8
qr_alg = QR(m)

r = qr_alg.H.copy()
n = r.shape[0]
iter_ = 100
subdiagonal_elements = list()

print("Convergence for the Wilkinson shift.")
print(f"Query matrix\n {pd.DataFrame(m)}")
print(f"Above matrix is converted to hessenberg form when QR(m) is initialised.")
print(f"Hessenberg form of the matrix {pd.DataFrame(qr_alg.H)}.")
print(f"Performing {iter_} iterations.")
print(f"Starting...\n")

for i in range(iter_):
	# sigma_k = qr_alg.wilkinson_shift(r[n - 2 :, n - 2 :])		
	# Generate a scaled identity matrix for use in the shifts.
	shift_mat = h[n - 1, n - 1] * np.eye(n, dtype = r.dtype)
 
	r -= shift_mat
	# Perform a step of the QR 
	# hessenberg method.
	q, r = qr_alg.qr_hessenberg(r)
	r += shift_mat
	# print(f"Submatrix \n {pd.DataFrame(r[n - 2 :, n - 2 :])}\n")
	subdiagonal_elements.append([abs(r[n - 1, n - 2])])
 
for i in range(1, len(subdiagonal_elements)):
    subdiagonal_elements[i].append(subdiagonal_elements[i][0] / subdiagonal_elements[i - 1][0])
    
print(f"Ratio (as explained above): {np.mean(np.array(subdiagonal_elements[1:], dtype = np.float64), axis = 0)[1]}")
	

Convergence for the Wilkinson shift.
Query matrix
                 0               1               2               3               4
0  0.378974438821  0.383951514428  0.355279371794  0.861974558176  0.660416145073
1  0.265448581905  0.249648068856  0.727177180496  0.608317333744  0.826422742058
2  0.553861402523  0.204745464611  0.994639411330  0.457542033691  0.995244863472
3  0.947843884728  0.518966425652  0.209057363335  0.906425326918  0.293070212917
4  0.184365752792  0.539536998894  0.394161843026  0.783541363274  0.264402726093
Above matrix is converted to hessenberg form when QR(m) is initialised.
Hessenberg form of the matrix                 0               1               2               3               4
0  0.378974438821 -1.081337992556  0.383906607940 -0.366207556689  0.044807462860
1 -1.144388117493  1.766529094028 -0.703764029791  0.241156707912 -0.147273231665
2  0.000000000000 -1.081977963722  0.603066934090 -0.004223510484  0.356960594832
3  0.000000000000  0.000000